In [35]:
from keras.models import load_model

model = load_model("other_models/bestmodel.h5")

In [24]:
import pandas as pd
import numpy as np

team_data = pd.read_csv("data/teams_data.csv").filter(regex='^(?!Unnamed).+$')
team_data["TeamID"] = team_data["TeamID"].astype(np.int64)
team_data["Season"] = team_data["Season"].astype(np.int64)

In [25]:
team_data = team_data[team_data["Season"] == 2019]

In [26]:
seeds = pd.read_csv("data/DataFiles/NCAATourneySeeds.csv")
seeds = seeds[seeds["Season"] == 2019]

In [38]:
def process_data(data):
    x = data[["T0OrdinalRank", "T1OrdinalRank", "T03ptpct", "T0APG", "T0APG_diff", "T0BPG", "T0BPG_diff", "T0DRPG", "T0FPG_diff", "T0PPG_diff", "T0SPG", "T13ptpct", "T1APG", "T1APG_diff", "T1BPG", "T1BPG_diff", "T1DRPG", "T1FPG_diff", "T1PPG_diff", "T1SPG"]]
#     x = data.drop(['Season', 'Team1ID', 'Team2ID', "T0TeamName", "T1TeamName"], axis=1)
    return x

In [28]:
team_names = pd.read_csv("data/DataFiles/Teams.csv")

team_names = team_names[["TeamID", "TeamName"]]

In [29]:
team_data = team_data.merge(team_names, how="left", on="TeamID")

In [30]:
team_data.columns

Index(['2ptpct', '3ptpct', 'APG', 'APG_diff', 'BPG', 'BPG_diff', 'DRPG',
       'DRPG_diff', 'FPG', 'FPG_diff', 'FTpct', 'ORPG', 'ORPG_diff', 'PPG',
       'PPG_diff', 'RPG', 'RPG_diff', 'SPG', 'SPG_diff', 'Season', 'TeamID',
       'TeamName'],
      dtype='object')

In [31]:
games = pd.read_csv("data/DataFiles/NCAATourneySlots.csv")

In [32]:
games = games[games["Season"] == 2019]

In [36]:
def get_winner(team1, team2):
    
    ordinals = pd.read_csv('data/MasseyOrdinals/MasseyOrdinals.csv')

    ordinals = ordinals.sort_values(by='RankingDayNum')

    ordinals = ordinals.loc[ordinals['SystemName'] == 'POM']
    ordinals = ordinals.loc[ordinals["Season"] == 2019]
    
    team1_KP = ordinals.loc[ordinals["TeamID"] == team1, "OrdinalRank"].values[-1]
    team2_KP = ordinals.loc[ordinals["TeamID"] == team2, "OrdinalRank"].values[-1]
    
    game_data = {
        "Season": [2019],
        "Team1ID": [team1],
        "Team2ID": [team2],
        "T0OrdinalRank": [team1_KP],
        "T1OrdinalRank": [team2_KP],
    }
    
    games = pd.DataFrame.from_dict(game_data)
    
    merged = games.merge(team_data, how='left', left_on=['Team1ID', 'Season'], right_on=['TeamID', 'Season'])
    merged = merged.drop('TeamID', axis=1)
    merged = merged.rename(index=str, columns=lambda x: x if x in {'Season', 'Team1ID', 'Team2ID'} or x.startswith('T0') or x.startswith('T1') else 'T0' + x)

    merged = merged.merge(team_data, how='left', left_on=['Team2ID', 'Season'], right_on=['TeamID', 'Season'])
    merged = merged.drop('TeamID', axis=1)
    merged = merged.rename(index=str, columns=lambda x: x if x in {'Season', 'Team1ID', 'Team2ID'} or x.startswith('T0') or x.startswith('T1') else 'T1' + x)

    x = process_data(merged)
    
    return np.round(model.predict(x))
    

In [34]:
def get_team_name(team):
    return team_data.loc[team_data["TeamID"] == team, "TeamName"].values[0]

In [39]:
import random
for index, row in games.iterrows():
    teams = [row["StrongSeed"], row["WeakSeed"]]
    random.shuffle(teams)
    team1 = seeds.loc[seeds["Seed"] == teams[0], "TeamID"].values[0]
    team2 = seeds.loc[seeds["Seed"] == teams[1], "TeamID"].values[0]
    
    winner = team1 if get_winner(team1, team2) == 0 else team2
    print("{} vs {}".format(get_team_name(team1), get_team_name(team2)))
    print("{} wins!".format(get_team_name(winner)))
    seeds = seeds.append({ "Season": 2019, "Seed": row["Slot"], "TeamID": winner }, ignore_index=True)

Temple vs Belmont
Belmont wins!
N Dakota St vs NC Central
N Dakota St wins!
St John's vs Arizona St
St John's wins!
F Dickinson vs Prairie View
F Dickinson wins!
N Dakota St vs Duke
Duke wins!
Michigan St vs Bradley
Michigan St wins!
LSU vs Yale
LSU wins!
Virginia Tech vs St Louis
Virginia Tech wins!
Mississippi St vs Liberty
Mississippi St wins!
Belmont vs Maryland
Maryland wins!
Minnesota vs Louisville
Louisville wins!
UCF vs VA Commonwealth
VA Commonwealth wins!
Gardner Webb vs Virginia
Virginia wins!
Tennessee vs Colgate
Tennessee wins!
Purdue vs Old Dominion
Purdue wins!
Kansas St vs UC Irvine
Kansas St wins!
Wisconsin vs Oregon
Wisconsin wins!
St Mary's CA vs Villanova
Villanova wins!
Cincinnati vs Iowa
Cincinnati wins!
Mississippi vs Oklahoma
Mississippi wins!
North Carolina vs Iona
North Carolina wins!
Abilene Chr vs Kentucky
Kentucky wins!
Georgia St vs Houston
Houston wins!
Northeastern vs Kansas
Kansas wins!
New Mexico St vs Auburn
Auburn wins!
Iowa St vs Ohio St
Iowa St win

In [17]:
import pickle

with open("logistic_regression/model.pickle", "rb") as f:
    model2 = pickle.load(f)

In [20]:
import random
for index, row in games.iterrows():
    teams = [row["StrongSeed"], row["WeakSeed"]]
    random.shuffle(teams)
    team1 = seeds.loc[seeds["Seed"] == teams[0], "TeamID"].values[0]
    team2 = seeds.loc[seeds["Seed"] == teams[1], "TeamID"].values[0]
    
    winner = team1 if get_winner(team1, team2) == 0 else team2
    print("{} vs {}".format(get_team_name(team1), get_team_name(team2)))
    print("{} wins!".format(get_team_name(winner)))
    seeds = seeds.append({ "Season": 2019, "Seed": row["Slot"], "TeamID": winner }, ignore_index=True)

Belmont vs Temple
Belmont wins!
N Dakota St vs NC Central
N Dakota St wins!
St John's vs Arizona St
Arizona St wins!
F Dickinson vs Prairie View
F Dickinson wins!
Duke vs N Dakota St
Duke wins!
Bradley vs Michigan St
Michigan St wins!
Yale vs LSU
LSU wins!
St Louis vs Virginia Tech
Virginia Tech wins!
Mississippi St vs Liberty
Liberty wins!
Maryland vs Belmont
Belmont wins!
Minnesota vs Louisville
Louisville wins!
UCF vs VA Commonwealth
UCF wins!
Gardner Webb vs Virginia
Virginia wins!
Tennessee vs Colgate
Tennessee wins!
Old Dominion vs Purdue
Purdue wins!
Kansas St vs UC Irvine
Kansas St wins!
Oregon vs Wisconsin
Wisconsin wins!
Villanova vs St Mary's CA
Villanova wins!
Iowa vs Cincinnati
Cincinnati wins!
Oklahoma vs Mississippi
Mississippi wins!
Iona vs North Carolina
North Carolina wins!
Abilene Chr vs Kentucky
Kentucky wins!
Georgia St vs Houston
Houston wins!
Northeastern vs Kansas
Kansas wins!
New Mexico St vs Auburn
Auburn wins!
Ohio St vs Iowa St
Iowa St wins!
Seton Hall vs Wo